In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [7]:
# tokenizer = AutoTokenizer.from_pretrained("BarraHome/Mistroll-7B-v2.2")
# model = AutoModelForCausalLM.from_pretrained("BarraHome/Mistroll-7B-v2.2")

# tokenizer = AutoTokenizer.from_pretrained("pansophic/rocket-3B")
# model = AutoModelForCausalLM.from_pretrained("pansophic/rocket-3B")

tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# prompt = "Hey, are you conscious? Can you talk to me?"
# inputs = tokenizer(prompt, return_tensors="pt")
# 
# generate_ids = model.generate(inputs.input_ids, max_length=30)
# tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [4]:
# from transformers import set_seed

# set_seed(0)
# model.to(device)

In [8]:
prompt = "Hey, are you conscious? Can you talk to me?"
model_inputs = tokenizer(prompt, return_tensors="pt")#.to(model.device)

In [9]:
generated_ids = model.generate(**model_inputs, max_length=32, do_sample=True)
responses = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
responses

'Hey, are you conscious? Can you talk to me?\n\nBrad: (coughs) Yeah, I can hear you.\n\n'

In [47]:
file_path = "./examples/example1.py"

In [48]:
flake8_out = !flake8 --format='%(row)d,%(col)d::%(code)s::%(text)s' "$file_path"

for i in flake8_out:
    print(i)
    
flake8_out = '\n'.join(flake8_out)

5,1::E305::expected 2 blank lines after class or function definition, found 1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [49]:
mypy_out = !mypy "$file_path" --no-color-output --pretty --strict 

for i in mypy_out:
    print(i)
    
mypy_out = '\n'.join(mypy_out)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


examples/example1.py:1: error: Function is missing a type annotation 
[no-untyped-def]
    def c(a, b):
    ^
examples/example1.py:7: error: Call to untyped function "c" in typed context 
[no-untyped-call]
    result = c(x, y)
             ^~~~~~~
Found 2 errors in 1 file (checked 1 source file)


In [50]:
with open(file_path, 'r') as f:
    code = f.read()

for i in code.split('\n'):
    print(i)

def c(a, b):
    s = a + b
    return s

x = 5
y = 10
result = c(x, y)
print("Result:", result)



In [56]:
prompt = "Suggest refactoring of the following code:\n" + code + "\nbased on the analysis output in format row,column::error_code::error:\n" + flake8_out + "\nand the analysis of this output:\n" + mypy_out
for i in prompt.split('\n'):
    print(i)
model_inputs = tokenizer(prompt, return_tensors="pt")

Suggest refactoring of the following code:
def c(a, b):
    s = a + b
    return s

x = 5
y = 10
result = c(x, y)
print("Result:", result)

based on the analysis output in format row,column::error_code::error:
5,1::E305::expected 2 blank lines after class or function definition, found 1
and the analysis of this output:
examples/example1.py:1: error: Function is missing a type annotation 
[no-untyped-def]
    def c(a, b):
    ^
examples/example1.py:7: error: Call to untyped function "c" in typed context 
[no-untyped-call]
    result = c(x, y)
             ^~~~~~~
Found 2 errors in 1 file (checked 1 source file)


In [59]:
generated_ids = model.generate(**model_inputs, max_length=1024, do_sample=True)
responses = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

KeyboardInterrupt: 

In [ ]:
for i in responses.split('\n'):
    print(i)